In [1]:
import sys
sys.path.insert(0, "..")
from gpbp_osm.layers import AdmArea
from gpbp_osm import visualisation
from gpbp_osm.distance import calculate_isopolygons_graph, calculate_isopolygons_Mapbox, get_isopolygons_gdf, population_served
from gpbp_osm.utils import group_population

import osmnx as ox
import warnings
warnings.filterwarnings(action='ignore')

### Administrative Area

In [2]:
adm_area = AdmArea(country="Greece", level=2)

Retrieving data for Greece of granularity level 2
Administrative areas for level 2:
['North Aegean' 'South Aegean' 'Athos' 'Attica' 'Crete' 'Epirus'
 'West Macedonia' 'Central Macedonia' 'East Macedonia and Thrace'
 'Ionian Islands' 'Peloponnese' 'West Greece' 'Central Greece' 'Thessaly']


In [3]:
adm_area.get_adm_area(adm_name="Crete")

In [4]:
adm_area.get_facilities(method="osm", tags={"building":"hospital"})
visualisation.plot_facilities(adm_area.fac_gdf)

Retrieving {'building': 'hospital'} for Crete area


In [5]:
adm_area.get_population(method="world_pop")
#visualisation.plot_population_heatmap(adm_area.pop_df)

Data downloaded
Converting raster file to dataframe


In [5]:
adm_area.pop_df

,longitude,latitude,population
0,23.736667,35.695833,0.067376
1,23.737500,35.695833,0.070566
2,23.738333,35.695833,0.072878
3,23.735833,35.695000,0.068367
4,23.736667,35.695000,0.075395
...,...,...,...
1203050,24.121667,34.801667,0.016128
1203051,24.122500,34.801667,0.015183
1203052,24.120833,34.800833,0.016218
1203053,24.121667,34.800833,0.015266


In [5]:
adm_area.compute_potential_fac(spacing=0.05)
visualisation.plot_facilities(adm_area.grid_gdf)

In [7]:
road_network = ox.graph_from_polygon(adm_area.geometry, network_type = "drive")

In [8]:
location_coords = adm_area.fac_gdf.iloc[0].geometry.coords
location_coords = list(location_coords)[0]

In [9]:
bounding_polys = calculate_isopolygons_Mapbox(
    location_coords, "driving", "distance", [2000, 5000, 10000])

https://api.mapbox.com/isochrone/v1/mapbox/driving/24.1246768,35.5174839?contours_meters=2000,5000,10000&polygons=true&denoise=1&access_token=pk.eyJ1Ijoia291c2FlIiwiYSI6ImNsYWk1cXRjazA2dGIzbnBiMjZyODk4ZWYifQ.4nvPS7fL7YtN4jcex9B5vg


In [10]:
bounding_polys

{'10000': <shapely.geometry.multipolygon.MultiPolygon at 0x1cc8c3a86d0>,
 '5000': <shapely.geometry.multipolygon.MultiPolygon at 0x1cc3f89d000>,
 '2000': <shapely.geometry.multipolygon.MultiPolygon at 0x1cc3f89c460>}

In [13]:
visualisation.plot_isochrone(bounding_polys["2000"])

In [11]:
bounding_poly_osm = calculate_isopolygons_graph(road_network, location_coords, "length", [2000, 5000, 10000])

In [12]:
bounding_poly_osm

{'2000': <shapely.geometry.polygon.Polygon at 0x1cc90753190>,
 '5000': <shapely.geometry.polygon.Polygon at 0x1cc90753940>,
 '10000': <shapely.geometry.polygon.Polygon at 0x1cc90750730>}

In [14]:
visualisation.plot_isochrone(bounding_poly_osm["2000"])

In [6]:
iso_gdf = get_isopolygons_gdf(
    adm_area.fac_gdf,
    "driving",
    "distance",
    [2000],
    "2000",
)

In [7]:
iso_gdf

,longitude,latitude,geometry
osmid,,,
6786674139,24.124677,35.517484,"MULTIPOLYGON (((24.12668 35.53296, 24.12659 35..."
13276279,24.015531,35.478433,"MULTIPOLYGON (((24.01653 35.48995, 24.01644 35..."
571736158,24.886137,35.290849,"MULTIPOLYGON (((24.88614 35.30197, 24.88414 35..."
221514762,25.083120,35.303893,"MULTIPOLYGON (((25.07412 35.31501, 25.07373 35..."
354942025,25.154615,35.305851,"MULTIPOLYGON (((25.15161 35.31915, 25.15002 35..."
25049463,25.713491,35.194360,"MULTIPOLYGON (((25.71249 35.21043, 25.71194 35..."
371354658,25.481251,35.201906,"MULTIPOLYGON (((25.47925 35.20901, 25.47862 35..."
371354659,25.481665,35.201820,"MULTIPOLYGON (((25.47866 35.20906, 25.47818 35..."
371354660,25.481653,35.202082,"MULTIPOLYGON (((25.47865 35.20914, 25.47860 35..."


In [8]:
serve_dict = population_served(iso_gdf, adm_area.pop_df, "osmid", "ID", nof_digits=3)

In [9]:
serve_dict

{13276279: [146287,
  146288,
  147875,
  146602,
  149813,
  146919,
  147556,
  147238,
  146920,
  146603,
  147557,
  147876,
  147239,
  146604,
  147558,
  146921,
  145643,
  146284,
  145966,
  146285,
  146286,
  148192,
  149161,
  147553,
  146599,
  149810,
  149485,
  148837,
  146916,
  147872,
  147554,
  148838,
  146600,
  148515,
  147873,
  149162,
  149811,
  148193,
  146917,
  147235,
  149486,
  147236,
  146918,
  146601,
  148516,
  149163,
  147555,
  149487,
  148194,
  147874,
  148839,
  149812,
  147237,
  150138,
  145963,
  145321,
  145641,
  144362,
  146282,
  145642,
  145322,
  145964,
  144363,
  146283,
  145965,
  144683,
  145323,
  145003,
  144364,
  150132,
  149482,
  146913,
  150456,
  149807,
  148834,
  147869,
  149158,
  151099,
  147231,
  147550,
  148511,
  146597,
  149159,
  148835,
  150133,
  147870,
  150457,
  146914,
  147232,
  149808,
  150778,
  149483,
  148512,
  147551,
  148190,
  150458,
  146915,
  149484,
  146598,
